In [1]:
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Dense
import numpy as np
import os
from keras.models import load_model

import numpy as np
from tensorflow.keras import layers, regularizers, initializers, constraints
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.layers import *
from keras_nlp import Glove
import torch

from scipy.special import softmax

C:\Users\uc_ma\anaconda3\envs\SciARK\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
Context_encoder = load_model('trained_models/bert_based_context_encoder.h5')
print("Loaded model from disk")

Loaded model from disk


In [3]:
from transformers import AutoTokenizer, AutoModel
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased', model_max_length=512)
bert_model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased').to(
        device)

In [1]:
#Inference Function
from infer_library import *
import numpy as np

Sentence_splitter = SentenceSplitter()

def infer_sent_labels(text):
    #Argument Labels.
    label2id = {0: 'PAD', 1: 'NONE', 2: 'EVIDENCE', 3: 'CLAIM'}

    #Split text in sentences
    text_splitted = Sentence_splitter.tokenize(text)

    #Encode text 
    text_encoded = [encode_text(sentence, bert_tokenizer, bert_model, device) for sentence in text_splitted]

    #Pad text
    max_sentences = 20
    text_encoded_padded = pad_array(np.stack(text_encoded), max_sentences) #2D Array

    #Create a tensor in order to have a valid input for Context encoder (#always 1 doc, #sentences, 768)
    tensor_final = np.expand_dims(text_encoded_padded, axis=0)

    #Make predictions for sentences --> type = ndarray
    predictions = Context_encoder.predict(tensor_final)
    
    #Reduce dimensions to 2
    reduced_predictions = np.squeeze(predictions)

    #Turn ndarray to list
    new_predictions_as_list = reduced_predictions.tolist()
    
    #Keep just valid sentences' scores and not padded scores i.e. "PAD" sents' scores
    final_predictions = new_predictions_as_list[20 - (len(text_splitted)):]

    #Run Softmax
    labels = softmax(predictions).argmax(-1)
    print(labels)

    #Return results
    labels_new = []
    for array in labels:
        for number in array:
            labels_new.append(label2id[number])

    labels_new_truncated = labels_new[20 - (len(text_splitted)):]
    results = dict(zip(text_splitted, labels_new_truncated))
    results["scores"] = final_predictions

    return results


# list1 = "Intraoperative mitomycin versus postoperative 5-fluorouracil in high-risk glaucoma filtering surgery. In a randomized clinical trial, the authors compared the use of postoperative subconjunctival injections of 5-fluorouracil (5-FU) in 19 eyes with a single intraoperative application of subconjunctival mitomycin (MMC) at the filtering site in 20 eyes at high risk for failure of glaucoma filtering surgery. Six months after surgery, intraocular pressures averaged 10.9 +/- 5.3 mmHg (mean +/- standard deviation) in the MMC-treated eyes versus 14.2 +/- 5.5 mmHg in the 5-FU-treated eyes (P = 0.08) and were less than or equal to 12 mmHg in 60.0% of MMC-treated eyes and 21.1% of 5-FU-treated eyes (P = 0.03). Mitomycin-treated eyes were receiving an average of 0.3 +/- 0.5 medications for intraocular pressure control, and 5-FU-treated eyes were receiving an average of 1.1 +/- 1.1 medications (P = 0.01). Drug-induced corneal epithelial defects were seen in nine 5-FU-treated eyes and in no MMC-treated eyes (P = 0.0004). These results suggest that intraoperative MMC may be a viable alternative to postoperative 5-FU, with lower overall intraocular pressures, decreased dependence on postoperative ocular antihypertensive medications, and decreased corneal toxicity."
# results = infer_sent_labels(list1)

# list2 = "Trabeculectomy with mitomycin. A comparative study with fluorouracil. We conducted a prospective, randomized study to compare mitomycin with fluorouracil for efficacy and safety as an adjunct to trabeculectomy in eyes with glaucoma with poor surgical prognosis. Thirty-two eyes of 32 patients were randomly allocated to either mitomycin (17 eyes) or fluorouracil therapy (15 eyes). Mitomycin (0.2 mg) was applied only once during trabeculectomy, and fluorouracil (5 mg) was subconjunctivally injected 10 times in the 2 weeks after surgery. Fifteen eyes (88%) in the mitomycin group and seven (47%) in the fluorouracil group achieved an intraocular pressure of less than or equal to 20 mm Hg without antiglaucoma medication. The follow-up period was 7 to 12 months. Corneal complications were less common in the group given mitomycin than in that given fluorouracil (12% vs 53%). The incidence of other complications was similar between the two groups."
# results2 = infer_sent_labels(list2)

C:\Users\uc_ma\anaconda3\envs\SciARK\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'bert_tokenizer' is not defined

In [16]:
list1 = "Intraoperative mitomycin versus postoperative 5-fluorouracil in high-risk glaucoma filtering surgery. In a randomized clinical trial, the authors compared the use of postoperative subconjunctival injections of 5-fluorouracil (5-FU) in 19 eyes with a single intraoperative application of subconjunctival mitomycin (MMC) at the filtering site in 20 eyes at high risk for failure of glaucoma filtering surgery. Six months after surgery, intraocular pressures averaged 10.9 +/- 5.3 mmHg (mean +/- standard deviation) in the MMC-treated eyes versus 14.2 +/- 5.5 mmHg in the 5-FU-treated eyes (P = 0.08) and were less than or equal to 12 mmHg in 60.0% of MMC-treated eyes and 21.1% of 5-FU-treated eyes (P = 0.03). Mitomycin-treated eyes were receiving an average of 0.3 +/- 0.5 medications for intraocular pressure control, and 5-FU-treated eyes were receiving an average of 1.1 +/- 1.1 medications (P = 0.01). Drug-induced corneal epithelial defects were seen in nine 5-FU-treated eyes and in no MMC-treated eyes (P = 0.0004). These results suggest that intraoperative MMC may be a viable alternative to postoperative 5-FU, with lower overall intraocular pressures, decreased dependence on postoperative ocular antihypertensive medications, and decreased corneal toxicity."
results = infer_sent_labels(list1)
print(results)

list2 = "Trabeculectomy with mitomycin. A comparative study with fluorouracil. We conducted a prospective, randomized study to compare mitomycin with fluorouracil for efficacy and safety as an adjunct to trabeculectomy in eyes with glaucoma with poor surgical prognosis. Thirty-two eyes of 32 patients were randomly allocated to either mitomycin (17 eyes) or fluorouracil therapy (15 eyes). Mitomycin (0.2 mg) was applied only once during trabeculectomy, and fluorouracil (5 mg) was subconjunctivally injected 10 times in the 2 weeks after surgery. Fifteen eyes (88%) in the mitomycin group and seven (47%) in the fluorouracil group achieved an intraocular pressure of less than or equal to 20 mm Hg without antiglaucoma medication. The follow-up period was 7 to 12 months. Corneal complications were less common in the group given mitomycin than in that given fluorouracil (12% vs 53%). The incidence of other complications was similar between the two groups."
results2 = infer_sent_labels(list2)
print(results2)

Please enter a string
1/1 [==============================] - 0s 15ms/step
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 2 2 3]]
{'Intraoperative mitomycin versus postoperative 5-fluorouracil in high-risk glaucoma filtering surgery.': 'NONE', 'In a randomized clinical trial, the authors compared the use of postoperative subconjunctival injections of 5-fluorouracil (5-FU) in 19 eyes with a single intraoperative application of subconjunctival mitomycin (MMC) at the filtering site in 20 eyes at high risk for failure of glaucoma filtering surgery.': 'NONE', 'Six months after surgery, intraocular pressures averaged 10.9 +/- 5.3 mmHg (mean +/- standard deviation) in the MMC-treated eyes versus 14.2 +/- 5.5 mmHg in the 5-FU-treated eyes (P = 0.08) and were less than or equal to 12 mmHg in 60.0% of MMC-treated eyes and 21.1% of 5-FU-treated eyes (P = 0.03).': 'NONE', 'Mitomycin-treated eyes were receiving an average of 0.3 +/- 0.5 medications for intraocular pressure control, and 5-FU-treated eyes were r

In [17]:
import json

with open('C:\\Users\\uc_ma\\Desktop\\Διπλωματική Εργασία NLP\\scientific_articles_to_infer_for_claim_detection\\food_pubs_to_infer_for_claim_detection.json', 'r') as d:
    dictData = json.load(d)
    print(type(dictData))

<class 'dict'>


In [19]:
#Inference routine

final_results = {}
from langdetect import detect
                
for i in dictData.keys(): #το doi είναι το i, το κρατάω
    exp_dict = {}
    if count <= 20: #number of texts to infer
        exp2_dict = {}
        if not isinstance(dictData[i], list):
            if dictData[i]["abstract"]:
                if detect(dictData[i]["abstract"]) == "en": 
                    for key, value in dictData[i].items():
                        exp2_dict.update({key:value})
                    exp_dict[i] = exp2_dict  
                    exp_dict[i]["sentences"] = infer_sent_labels(exp_dict[i]["abstract"])
                    final_results.update(exp_dict)
                    count += 1

1/1 [==============================] - 0s 19ms/step
[[0 0 0 0 0 0 0 0 0 1 1 1 1 2 1 2 1 2 1 3]]
1/1 [==============================] - 0s 19ms/step
[[0 0 0 0 1 1 1 1 1 1 1 1 1 2 1 1 1 1 3 1]]
1/1 [==============================] - 0s 22ms/step
[[0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 3]]
1/1 [==============================] - 0s 19ms/step
[[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 3]]
1/1 [==============================] - 0s 24ms/step
[[0 0 0 0 0 0 1 1 1 1 1 1 1 1 2 2 2 2 1 3]]
1/1 [==============================] - 0s 23ms/step
[[0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 2 2 1 3 3]]
1/1 [==============================] - 0s 22ms/step
[[0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 2 1 1 1 3]]
1/1 [==============================] - 0s 17ms/step
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1]]
1/1 [==============================] - 0s 19ms/step
[[0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1]]
1/1 [==============================] - 0s 19ms/step
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1]]
1/1 [==============================] - 0

In [20]:
with open("with_scores_sample_food.json", "w") as outfile:
    json.dump(final_results, outfile)